In [ ]:
#————————————————————

# Name: Azure OpenAI Assistant for Nutritional Values (V1)

# Purpose:


# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 22.01.2024
# Python Version: 3.10.4

#General Sources:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant
# https://learn.microsoft.com/en-us/azure/ai-services/openai/assistants-quickstart?tabs=command-line&pivots=programming-language-studio

#Azure Openai Usage:
# https://stackoverflow.com/questions/77986927/in-azure-openai-assistants-when-i-upload-a-file-and-save-it-where-is-that-file-s
# https://techcommunity.microsoft.com/t5/fasttrack-for-azure/strategies-for-optimizing-high-volume-token-usage-with-azure/ba-p/4007751#:~:text=Understanding%20tokens%20and%20limits%20in,generation%2C%20translation%2C%20or%20summarization.

#Additionals:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models

# Download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [13]:
# Import required libraries
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI

In [2]:
# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
load_dotenv(dotenv_path=Path("C:\Python\openai-lab\.venv\.env"))
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [3]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = azure_oai_key,  
    api_version = "2024-02-15-preview",
    azure_endpoint = azure_oai_endpoint
    )

In [4]:
# Load data
path_input = r"C:\Python\data\openfoodfacts.xlsx" #Change path if required

# send the csv file to the assistant purpose files
response = client.files.create(
  file=open(path_input, "rb"),
  purpose="assistants"
)
print(response)
file__id = response.id

FileObject(id='assistant-RfxYH5aPCpuEIr4jaZXA5MQk', bytes=18276416, created_at=1710246487, filename='openfoodfacts.xlsx', object='file', purpose='assistants', status='processed', status_details=None)


In [5]:
instructions = '''
You are a senior data analyst who will work with data in an xlsx file.
You have access to a sandboxed environment for writing python code.
When the user asks you to perform your actions, you will use the provided xlsx file.
You will perform data cleansing and transformation steps.
Execute each of the steps listed below in your ACTIONS section.

ACTIONS:

1. Read the xlsx file into a pandas DataFrame.
2. Keep only the columns "product_name", "level_1", "level_2", "level_3".
3. Trim and lowercase the values of columns "product_name", "level_1", "level_2", "level_3".
4. Remove rows with non-roman character such as Arabic, Chinese, Cyrillic, Greek, Hebrew, Japanese, Korean, Tamil and Thai from columns "product_name", "level_1", "level_2", "level_3".
5. Remove rows with missing, empty or NA values from columns "product_name", "level_1", "level_2", "level_3".
6. Remove duplicate values from column "product_name" and prepare the results as Table_1.
7. Prepare Table_1 as an xlsx file for download by the user. 
8. Provide a summary paragraph explaining the preparation of the data set.

DO NOT:
1. Do not return any images. 
2. Do not return any other file types.
'''

In [6]:
# Create an assistant
assistant = client.beta.assistants.create(
    name = "data analyst assistant",
    instructions = instructions,
    tools = [{"type": "code_interpreter"}],
    model = "gpt-4-1106-preview", #You must replace this value with the deployment name for your model.
    file_ids=[file__id]
)

In [7]:
# Get the file id
fileId = assistant.file_ids[0]
print(assistant)

Assistant(id='asst_JXiCvERyfKilB6aAh4Eu9Pm3', created_at=1710246506, description=None, file_ids=['assistant-RfxYH5aPCpuEIr4jaZXA5MQk'], instructions='\nYou are a senior data analyst who will work with data in an xlsx file.\nYou have access to a sandboxed environment for writing python code.\nWhen the user asks you to perform your actions, you will use the provided xlsx file.\nYou will perform data cleansing and transformation steps.\nExecute each of the steps listed below in your ACTIONS section.\n\nACTIONS:\n\n1. Read the xlsx file into a pandas DataFrame.\n2. Keep only the columns "product_name", "level_1", "level_2", "level_3".\n3. Trim and lowercase the values of columns "product_name", "level_1", "level_2", "level_3".\n4. Remove non-alphanumeric characters from column "product_name".\n5. Remove empty or NA rows from columns "product_name", "level_1", "level_2", "level_3".\n6. Remove duplicate values from column "product_name" and prepare the results as Table_1.\n7. Prepare Table_1

In [8]:
# Create a thread
thread = client.beta.threads.create()

In [9]:
# Add a user prompt to the thread

prompt = "Please execute your ACTIONS on the data stored in the xlsx file " + fileId

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = prompt
)

In [10]:
# Run the Assistant

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [11]:
while True:
    sec = 30
    # Wait for 30 seconds
    time.sleep(sec)  
    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            try:
                content = msg.content[0].text.value
                print(f"{role.capitalize()}: {content}")
            except AttributeError:
                # This will execute if .text does not exist
                print(f"{role.capitalize()}: [Non-text content, possibly an image or other file type]")
        break
    elif run.status == "requires_action":
        # handle function calling and continue with the execution
        pass
    elif run.status == "expired" or run.status=="failed" or run.status=="cancelled":
        # run failed, expired, or was cancelled
        break    
    else:
        print("in progress...")

in progress...
in progress...
in progress...
in progress...
Assistant: The data set preparation is complete. Here's a summary of the steps performed:

1. I read the provided Excel file into a pandas DataFrame.
2. I retained only the relevant columns: "product_name", "level_1", "level_2", and "level_3".
3. I trimmed whitespace and converted the string values to lowercase in these columns to ensure consistency.
4. I removed any non-alphanumeric characters from the "product_name" column to clean the product names.
5. I discarded rows with missing or NA values in any of the four columns to maintain data integrity.
6. I eliminated duplicate entries from the "product_name" column to ensure that each product name is unique in the final table, now referred to as Table_1.
7. I have prepared Table_1 as an xlsx file ready for your download:

[Download the cleaned product data xlsx file](sandbox:/mnt/data/cleaned_product_data.xlsx)

If you require any further analysis or modifications, please let 

In [19]:
# Functions to read xlsx files from Azure Openai

def read_and_save_file(first_file_id, file_name):    
    # its binary, so read it and then make it a file like object
    file_data = client.files.content(first_file_id)
    file_data_bytes = file_data.read()
    file_like_object = io.BytesIO(file_data_bytes)
    #now read as csv to create df
    returned_data = pd.read_excel(file_like_object)
    returned_data.to_excel(file_name, index=False)
    return returned_data
    # file = read_and_save_file(first_file_id, "analyst_output.csv")
    
def files_from_messages(messages, asst_name):
    first_thread_message = messages.data[0]  # Accessing the first ThreadMessage
    message_ids = first_thread_message.file_ids
    print(message_ids)
    # Loop through each file ID and save the file with a sequential name
    for i, file_id in enumerate(message_ids):
        file_name = f"{asst_name}_output_{i+1}.xlsx"  # Generate a sequential file name
        read_and_save_file(file_id, file_name)
        print(f'saved {file_name}')  

In [ ]:
# extract the file names from the response and retrieve the content
asst_name = 'data_analyst_assistant'        
files_from_messages(messages, asst_name)

In [ ]:
#Clean up

client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)
for i in client.files.list():
    client.files.delete(i.id)

In [ ]:

for i in client.beta.assistants.list():
    client.beta.assistants.delete(i.id)
for i in client.beta.threads.list():
    client.beta.threads.delete(i.id)
for i in client.files.list():
    client.files.delete(i.id)

In [25]:
#Test run 50 people

for i in range(50):

    # Load data
    path_input = r"C:\Python\data\openfoodfacts.xlsx" #Change path if required

    # send the csv file to the assistant purpose files
    response = client.files.create(
    file=open(path_input, "rb"),
    purpose="assistants"
    )

    file__id = response.id

    # Create an assistant
    assistant = client.beta.assistants.create(
        name = "data analyst assistant " + str(i),
        instructions = instructions,
        tools = [{"type": "code_interpreter"}],
        model = "gpt-4-1106-preview", #You must replace this value with the deployment name for your model.
        file_ids=[file__id]
    )

    fileId = assistant.file_ids[0]

    thread = client.beta.threads.create()

    prompt = "Please execute your ACTIONS on the data stored in the xlsx file " + fileId

    message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = prompt
    )

    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
    )
    